In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import time
import snscrape.modules.twitter as sntwitter
# !pip install snscrape

In [ ]:
# defining the search parameters and limits for the scraping package to adhere to
query = '(abortion OR abortion laws OR pro life OR pro choice) lang:en until:2023-01-31 since:2022-01-01'
tweets = []
limit = 3000

# scraping and extracting the tweets with required parameters
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.user.username, tweet.content])
       
# storing the tweets in a dataframe
df = pd.DataFrame(tweets, columns = ['Date', 'User', 'Tweet']).astype(str)
df

/var/folders/rx/scdn4qfn2x9bfbkh3wgnqhm80000gn/T/ipykernel_1037/3763086.py:11: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.user.username, tweet.content])


,Date,User,Tweet
0,2023-01-30 23:56:38+00:00,Lord_Bogan,@radfembarbie @Slatzism @barneyxbt @number41_ ...
1,2023-01-30 23:49:39+00:00,saltyoldRN,"@DemforDeSantis Pro vaccine is one thing, mand..."
2,2023-01-30 23:44:51+00:00,JohnGil30224361,@stellalimelight @ACTBrigitte Old style pro ch...
3,2023-01-30 23:36:55+00:00,bzawi619,@eat_your_lasers @JoeBiden The court basically...
4,2023-01-30 23:35:23+00:00,sahbdeane,"@masterkiarie @AbbyJohnson Um. No.\n\nI am, an..."
...,...,...,...
2995,2022-12-26 05:08:17+00:00,ShaneDemont,@Aiyah02436291 @FreeDemizer @StudentsforLife s...
2996,2022-12-26 03:41:03+00:00,kreativeone6,@AshGren65006874 @prolifenanaki22 @AliceKing69...
2997,2022-12-26 02:59:06+00:00,Severed_Dad,@patrickbetdavid I got the vaccine but the thi...
2998,2022-12-26 02:16:45+00:00,TheRadiantSky1,@ed24gar @CT0197 @Jak4691 pro abortion implies...


In [ ]:
def preprocess(sentence):
    words = []
    for word in sentence.split():
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        elif word.startswith('http'):
            word = 'http'
        words.append(word)
    tweets_processed = " ".join(words)
    return tweets_processed

df['tweet_processed'] = df.apply(lambda x: preprocess(x.Tweet), axis = 1)
df.head()

,Date,User,Tweet,tweet_processed
0,2023-01-30 23:56:38+00:00,Lord_Bogan,@radfembarbie @Slatzism @barneyxbt @number41_ ...,@user @user @user @user I guess it comes from ...
1,2023-01-30 23:49:39+00:00,saltyoldRN,"@DemforDeSantis Pro vaccine is one thing, mand...","@user Pro vaccine is one thing, mandating is a..."
2,2023-01-30 23:44:51+00:00,JohnGil30224361,@stellalimelight @ACTBrigitte Old style pro ch...,@user @user Old style pro choice republican? I...
3,2023-01-30 23:36:55+00:00,bzawi619,@eat_your_lasers @JoeBiden The court basically...,@user @user The court basically said that the ...
4,2023-01-30 23:35:23+00:00,sahbdeane,"@masterkiarie @AbbyJohnson Um. No.\n\nI am, an...","@user @user Um. No. I am, and always have been..."


In [ ]:
tweets = df
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(roberta)
labels = ['Negative', 'Neutral', "Positive"]
def encode(tweet):
    out = model(**tokenizer(tweet, return_tensors='pt'))
    return labels[np.argmax(softmax(out[0][0].detach()))]

tweets['sentiment'] = tweets.apply(lambda x: encode(x.tweet_processed), axis = 1)
tweets['sentiment'].value_counts()

Negative    2151
Neutral      809
Positive      40
Name: sentiment, dtype: int64

In [ ]:
tweets.head()

,Date,User,Tweet,tweet_processed,sentiment
0,2023-01-30 23:56:38+00:00,Lord_Bogan,@radfembarbie @Slatzism @barneyxbt @number41_ ...,@user @user @user @user I guess it comes from ...,Negative
1,2023-01-30 23:49:39+00:00,saltyoldRN,"@DemforDeSantis Pro vaccine is one thing, mand...","@user Pro vaccine is one thing, mandating is a...",Neutral
2,2023-01-30 23:44:51+00:00,JohnGil30224361,@stellalimelight @ACTBrigitte Old style pro ch...,@user @user Old style pro choice republican? I...,Negative
3,2023-01-30 23:36:55+00:00,bzawi619,@eat_your_lasers @JoeBiden The court basically...,@user @user The court basically said that the ...,Negative
4,2023-01-30 23:35:23+00:00,sahbdeane,"@masterkiarie @AbbyJohnson Um. No.\n\nI am, an...","@user @user Um. No. I am, and always have been...",Positive


In [ ]:
tweets2 = tweets.drop('tweet_processed', axis = 1)
tweets2.to_csv('analyzed_tweets.csv', index = False)